In [1]:
val tableNames = List("Age_and_Gender", "Census_Business_Patterns", "Complex_Race", "Educational_Attainment", "Hispanic", "IRS_SOI_County", "Simple_Race")
val tables = for (name <- tableNames) yield (name, spark.read.parquet(s"s3://agimodeltrainer/Clean_Data/${name}.parquet"))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
22,application_1598467819421_0023,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tableNames: List[String] = List(Age_and_Gender, Census_Business_Patterns, Complex_Race, Educational_Attainment, Hispanic, IRS_SOI_County, Simple_Race)
tables: List[(String, org.apache.spark.sql.DataFrame)] = List((Age_and_Gender,[FIPS: string, Year: int ... 16 more fields]), (Census_Business_Patterns,[FIPS: string, Year: int ... 1 more field]), (Complex_Race,[FIPS: string, Year: int ... 27 more fields]), (Educational_Attainment,[FIPS: string, Year: int ... 13 more fields]), (Hispanic,[FIPS: string, Year: int ... 6 more fields]), (IRS_SOI_County,[FIPS: string, Year: int ... 2 more fields]), (Simple_Race,[FIPS: string, Year: int ... 6 more fields]))


In [2]:
tables.foreach(t => {
    println(t._1)
    println("Count: " + t._2.count)
    println("Count after na drop: " + t._2.na.drop.count)
    t._2.printSchema
})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Age_and_Gender
Count: 22543
Count after na drop: 22543
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Male: double (nullable = true)
 |-- Female: double (nullable = true)
 |-- Under_5: double (nullable = true)
 |-- 5_9: double (nullable = true)
 |-- 10_14: double (nullable = true)
 |-- 15_19: double (nullable = true)
 |-- 20_24: double (nullable = true)
 |-- 25_34: double (nullable = true)
 |-- 35_44: double (nullable = true)
 |-- 45_54: double (nullable = true)
 |-- 55_59: double (nullable = true)
 |-- 60_64: double (nullable = true)
 |-- 65_74: double (nullable = true)
 |-- 75_84: double (nullable = true)
 |-- 85plus: double (nullable = true)

Census_Business_Patterns
Count: 22338
Count after na drop: 22338
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Maps: map (nullable = true)
 |    |-- key: integer
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- ele

In [3]:
val tableIt = tables.toIterator
val colMap = (for (table <- tables) yield Map(table._1 -> table._2.columns.toSeq)).flatten.toMap
var joinedTable = tableIt.next._2
for (table <- tableIt) {
    joinedTable = joinedTable.join(table._2, Seq("FIPS", "Year"), "inner")
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tableIt: Iterator[(String, org.apache.spark.sql.DataFrame)] = non-empty iterator
colMap: scala.collection.immutable.Map[String,Seq[String]] = Map(Census_Business_Patterns -> WrappedArray(FIPS, Year, Maps), Complex_Race -> WrappedArray(FIPS, Year, One_Race, Only_White, Only_Black, Only_Native, Only_Cherokee, Only_Chippewa, Only_Navajo, Only_Sioux, Only_Asian, Only_Indian, Only_Chinese, Only_Filipino, Only_Japanese, Only_Korean, Only_Vietnamese, Only_Asian_Other, Only_Pacific, Only_Hawaiian, Only_Guamanian, Only_Samoan, Only_Pacific_Other, Only_Other, Twoplus_Races, White_Black, White_Native, White_Asian, Black_Native), Simple_Race -> WrappedArray(FIPS, Year, White, Black, Native, Asian, Pacific, Other), Hispanic -> WrappedArray(FIPS, Year, Hispanic, Mexican, Puerto_Rican, Cuban, Hispanic_Other, Not_Hispanic), Educational_Attainment -> WrappedArray(FIPS, Year, Pop_18_...joinedTable: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 16 more fields]


In [4]:
joinedTable.cache
joinedTable.count
joinedTable.na.drop.count
joinedTable.select("FIPS").distinct.count
joinedTable.select("Year").distinct.count
joinedTable.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [FIPS: string, Year: int ... 71 more fields]
res4: Long = 21972
res5: Long = 21972
res6: Long = 3144
res7: Long = 7
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Male: double (nullable = true)
 |-- Female: double (nullable = true)
 |-- Under_5: double (nullable = true)
 |-- 5_9: double (nullable = true)
 |-- 10_14: double (nullable = true)
 |-- 15_19: double (nullable = true)
 |-- 20_24: double (nullable = true)
 |-- 25_34: double (nullable = true)
 |-- 35_44: double (nullable = true)
 |-- 45_54: double (nullable = true)
 |-- 55_59: double (nullable = true)
 |-- 60_64: double (nullable = true)
 |-- 65_74: double (nullable = true)
 |-- 75_84: double (nullable = true)
 |-- 85plus: double (nullable = true)
 |-- Maps: map (nullable = true)
 |    |-- key: integer
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: integer (conta

In [5]:
for (m <- colMap) {
    println(m._1)
    m._2.foreach(c => if (c != "FIPS" && c != "Year") println(s"--- ${c.replace("_", " ")}"))
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Census_Business_Patterns
--- Maps
Complex_Race
--- One Race
--- Only White
--- Only Black
--- Only Native
--- Only Cherokee
--- Only Chippewa
--- Only Navajo
--- Only Sioux
--- Only Asian
--- Only Indian
--- Only Chinese
--- Only Filipino
--- Only Japanese
--- Only Korean
--- Only Vietnamese
--- Only Asian Other
--- Only Pacific
--- Only Hawaiian
--- Only Guamanian
--- Only Samoan
--- Only Pacific Other
--- Only Other
--- Twoplus Races
--- White Black
--- White Native
--- White Asian
--- Black Native
Simple_Race
--- White
--- Black
--- Native
--- Asian
--- Pacific
--- Other
Hispanic
--- Hispanic
--- Mexican
--- Puerto Rican
--- Cuban
--- Hispanic Other
--- Not Hispanic
Educational_Attainment
--- Pop 18 24
--- Less HS Grad 18 24
--- HS Grad 18 24
--- Some College 18 24
--- Bachelor 18 24
--- Pop 25plus
--- Less 9th 25plus
--- 9 12th 25plus
--- HS Grad 25plus
--- Some College 25plus
--- Associate 25plus
--- Bachelor 25plus
--- Gradplus 25plus
Age_and_Gender
--- Population
--- Male
--- Fe

In [6]:
sc.parallelize(colMap.toSeq).toDF.write.parquet("s3://agimodeltrainer/Clean_Data/ColumnMap.parquet")
joinedTable.write.parquet("s3://agimodeltrainer/Clean_Data/FinalTable.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
val coltest = spark.read.parquet("s3://agimodeltrainer/Clean_Data/ColumnMap.parquet")
coltest.as[(String, Seq[String])].collect.toMap.foreach(m => {
    println(m._1)
    m._2.foreach(c => {
        if (c != "FIPS" && c != "Year") println(s"--- $c")
    })
})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

coltest: org.apache.spark.sql.DataFrame = [_1: string, _2: array<string>]
Census_Business_Patterns
--- Maps
Complex_Race
--- One_Race
--- Only_White
--- Only_Black
--- Only_Native
--- Only_Cherokee
--- Only_Chippewa
--- Only_Navajo
--- Only_Sioux
--- Only_Asian
--- Only_Indian
--- Only_Chinese
--- Only_Filipino
--- Only_Japanese
--- Only_Korean
--- Only_Vietnamese
--- Only_Asian_Other
--- Only_Pacific
--- Only_Hawaiian
--- Only_Guamanian
--- Only_Samoan
--- Only_Pacific_Other
--- Only_Other
--- Twoplus_Races
--- White_Black
--- White_Native
--- White_Asian
--- Black_Native
Simple_Race
--- White
--- Black
--- Native
--- Asian
--- Pacific
--- Other
Hispanic
--- Hispanic
--- Mexican
--- Puerto_Rican
--- Cuban
--- Hispanic_Other
--- Not_Hispanic
Educational_Attainment
--- Pop_18_24
--- Less_HS_Grad_18_24
--- HS_Grad_18_24
--- Some_College_18_24
--- Bachelor_18_24
--- Pop_25plus
--- Less_9th_25plus
--- 9_12th_25plus
--- HS_Grad_25plus
--- Some_College_25plus
--- Associate_25plus
--- Bachelo

In [8]:
val t = spark.read.parquet("s3://agimodeltrainer/Clean_Data/FinalTable.parquet")
t.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 71 more fields]
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Male: double (nullable = true)
 |-- Female: double (nullable = true)
 |-- Under_5: double (nullable = true)
 |-- 5_9: double (nullable = true)
 |-- 10_14: double (nullable = true)
 |-- 15_19: double (nullable = true)
 |-- 20_24: double (nullable = true)
 |-- 25_34: double (nullable = true)
 |-- 35_44: double (nullable = true)
 |-- 45_54: double (nullable = true)
 |-- 55_59: double (nullable = true)
 |-- 60_64: double (nullable = true)
 |-- 65_74: double (nullable = true)
 |-- 75_84: double (nullable = true)
 |-- 85plus: double (nullable = true)
 |-- Maps: map (nullable = true)
 |    |-- key: integer
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- One_Race: double (nullable = true)
 |-- Only_White: double (nullable = true)
